In [2]:
tests_old = 'test_NHA.txt  test_RHC.txt  test_SA.txt  test_VOC.txt'.split()
# tests
tests = tests_old +['ned.testb'] # for the names


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, AutoModel
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
from data_utils import prepare_data, convert_to_dataset

# train and val are redundant but we need the labels, sooo
train = prepare_data('/ivi/ilps/personal/vprovat/KB/data/AITrainingset/Data/train.txt')
val = prepare_data('/ivi/ilps/personal/vprovat/KB/data/AITrainingset/Data/validation.txt')
tests_prepared = {test:
     prepare_data('/ivi/ilps/personal/vprovat/KB/data/AITrainingset/Data/'+test) for test in tests_old
                 } # NB: here it's a dictionary, used to be a list

tests_prepared['ned.testb'] = prepare_data('/ivi/ilps/personal/vprovat/KB/data/Dutch_conll/ned.testb')


  0%|          | 0/8040 [00:00<?, ?it/s]

  0%|          | 0/2150 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/5076 [00:00<?, ?it/s]

In [4]:
label_list = sorted(list(set([token_data[1] for sentence in train for token_data in sentence if token_data])))

label_list = label_list
# print(label_list_new)
id2label_original = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}
label_map = {label: i for i, label in enumerate(label_list)}
label_map.update({'B-ORG': label2id['O'],
                  'B-MISC': label2id['O'], 
                  'I-ORG': label2id['O'],
                  'I-MISC': label2id['O']})

train_data = convert_to_dataset(train, label_map)
val_data = convert_to_dataset(val, label_map)
test_data = [convert_to_dataset(test, label_map)
             for _, test in tests_prepared.items()]

In [5]:
id2label = id2label_original
dct = {
#     "train": train_data,
#     "validation": val_data # remove train and val for now
}
for i, test in enumerate(tests):
    dct[test.split('.')[0]] = test_data[i]

datasets = DatasetDict(dct)

In [6]:
import pickle
all_wrong_preds=pickle.load(open('all_wrong_preds.p','rb'))

In [7]:
# all_wrong_preds

In [8]:
# tests_prepared['test_RHC.txt']

In [9]:
def confused_per_and_loc(true_label, pred_label):
    return set([true_label.split('-')[-1],pred_label.split('-')[-1]]) == set(['PER','LOC'])

In [10]:
# wrong_pred = all_wrong_preds['GysBERT']['test_RHC.txt'][0]

In [11]:
# match_preds(wrong_pred)

In [12]:
model_names = {'GysBERT': "/ivi/ilps/personal/vprovat/KB/models/GysBERT-NER-v2",
              'BERTje': "/ivi/ilps/personal/vprovat/KB/models/BERTje-NER-v2",
              'BERT-multi-cased': "/ivi/ilps/personal/vprovat/KB/models/BERT-multi-cased-NER-v2",
              'WikiNEuRal': "Babelscape/wikineural-multilingual-ner"}

In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=True, return_tensors='pt'
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
def get_tokenized_datasets(model_name):
    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
    global model
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
    
    return tokenized_datasets

In [15]:
tokenized_datasets = {model_name: get_tokenized_datasets(model_path)
                     for model_name, model_path in model_names.items()}

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

Map:   0%|          | 0/5076 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

Map:   0%|          | 0/5076 [00:00<?, ? examples/s]

In [16]:
# NB: right now we're only interested in the PER vs LOC confusion
def match_entities(tokens, pred_labels, true_labels): # finds entities where predictions were wrong
    if len(tokens) != len(pred_labels): # long text was split into subsections
        print('OOPS! {0} != {1}'.format(len(tokens),len(pred_labels)))
#         print(tokens)
#         print(pred_labels)
        
    entities = []
    cur_tokens = []
    cur_true_labels = []
    cur_pred_labels = []
    cur_start = -1
    for i, token in enumerate(tokens):
#         if pred_labels[i] != true_labels[i]:
        if confused_per_and_loc(pred_labels[i],true_labels[i]):
            if not cur_tokens: # entity begins
                cur_start = i
            cur_tokens.append(token)
            cur_true_labels.append(true_labels[i])
            cur_pred_labels.append(pred_labels[i])
        else:
            if cur_tokens:
                entities.append((cur_start,
                                 cur_tokens,
                                cur_pred_labels,
                                cur_true_labels))
            cur_tokens = []
            cur_true_labels = []
            cur_pred_labels = []
            cur_start = -1
            
    if cur_tokens:
        entities.append((cur_start,
                         cur_tokens,
                                cur_pred_labels,
                                cur_true_labels))
    return entities

In [17]:
def match_preds(wrong_pred, model_name='GysBERT', test_name='test_RHC'):
    ind, pred_labels, true_labels = wrong_pred
        
    tokens = tokenized_datasets[model_name][test_name]['tokens'][ind] # check this just in case
#     tokenizer = AutoTokenizer.from_pretrained(model_names[model_name], model_max_length=512)
#     sentence = tokenizer.decode(tokenized_datasets[model_name][test_name]['input_ids'][ind])
    # todo update this
    try:
        return tokens,match_entities(tokens, pred_labels, true_labels)
    except:
        print('smth wrong, index ',ind)

In [18]:
#  toks = tokenized_datasets[MODEL]['test_NHA']['tokens'][22]

In [19]:
# preds = all_wrong_preds[MODEL][TEST][22][1]

In [20]:
from tqdm.auto import tqdm

In [21]:
# MODEL = 'GysBERT'
# TEST = 'test_NHA.txt'

records = []

for MODEL in model_names:
#     print('@@@@@@@')
#     print(MODEL)
#     print('@@@@@@@')
    for TEST in tests:
#         print(TEST)
        for i, wrong_pred in enumerate(tqdm(all_wrong_preds[MODEL][TEST])):
            try:
                tokens, preds = match_preds(wrong_pred, MODEL, TEST.split('.')[0])
            except:
                print('OOPS')
                continue
            if not preds:
                continue
            sentence = ' '.join(tokens)
        #     print(sentence)
#             print('====Sentence number {0}==='.format(i))
            for pred in preds:
                start, mention, pred_labels, true_labels = pred
#                 print(' '.join(tokens[:start]))
#                 print('+')
#                 print(' '.join(tokens[start:]))
#                 print('Start, mention, predicted, true')
#                 print(pred)

#                 print('______')
                
                records.append({'model_name': MODEL,
                               'test_set': TEST,
                                'prediction_number': i,
                                'sentence_number': all_wrong_preds[MODEL][TEST][i][0],
                                'left_context': ' '.join(tokens[:start]),
                                'right_context': ' '.join(tokens[start:]),
                                'mention_tokens': mention,
                                'predicted_labels': pred_labels,
                                'true_labels': true_labels
                               })

  0%|          | 0/39 [00:00<?, ?it/s]

KUT! 84 != 82
smth wrong, index  30
KUT
KUT! 418 != 406
smth wrong, index  49
KUT


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

KUT! 468 != 388
smth wrong, index  45
KUT
KUT! 408 != 378
smth wrong, index  66
KUT
KUT! 342 != 338
smth wrong, index  93
KUT


...

KUT! 293 != 248
smth wrong, index  10
KUT
KUT! 336 != 303
smth wrong, index  42
KUT
KUT! 468 != 318
smth wrong, index  45
KUT
KUT! 345 != 305
smth wrong, index  61
KUT
KUT! 408 != 323
smth wrong, index  66
KUT
KUT! 356 != 294
smth wrong, index  78
KUT
KUT! 326 != 289
smth wrong, index  88
KUT
KUT! 326 != 286
smth wrong, index  91
KUT
KUT! 288 != 262
smth wrong, index  92
KUT
KUT! 342 != 287
smth wrong, index  93
KUT
KUT! 350 != 288
smth wrong, index  95
KUT
KUT! 393 != 325
smth wrong, index  100
KUT


  0%|          | 0/111 [00:00<?, ?it/s]

KUT! 373 != 294
smth wrong, index  15
KUT
KUT! 229 != 228
smth wrong, index  33
KUT
KUT! 201 != 200
smth wrong, index  77
KUT
KUT! 267 != 265
smth wrong, index  91
KUT
KUT! 448 != 248
smth wrong, index  106
KUT
KUT! 284 != 262
smth wrong, index  107
KUT


  0%|          | 0/2976 [00:00<?, ?it/s]

KUT! 409 != 400
smth wrong, index  2007
KUT
KUT! 28 != 27
smth wrong, index  2434
KUT
KUT! 12 != 11
smth wrong, index  3108
KUT


In [22]:
import pandas as pd

df = pd.DataFrame.from_records(records)
df.to_csv('error_analysis_df.tsv',sep='\t')

In [23]:
df.sample()

,model_name,test_set,prediction_number,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
394,WikiNEuRal,ned.testb,2276,3863,(,FR ),[FR],[B-LOC],[B-PER]


In [24]:
records_per_loc = []

for MODEL in model_names:
#     print('@@@@@@@')
#     print(MODEL)
#     print('@@@@@@@')
    for TEST in tests:
#         print(TEST)
        for i, wrong_pred in enumerate(tqdm(all_wrong_preds[MODEL][TEST])):
            try:
                tokens, preds = match_preds(wrong_pred, MODEL, TEST.split('.')[0])
            except:
                print('OOPS')
                continue
            if not preds:
                continue
            sentence = ' '.join(tokens)
        #     print(sentence)
#             print('====Sentence number {0}==='.format(i))
            for pred in preds:
                start, mention, pred_labels, true_labels = pred
#                 print(' '.join(tokens[:start]))
#                 print('+')
#                 print(' '.join(tokens[start:]))
#                 print('Start, mention, predicted, true')
#                 print(pred)

#                 print('______')
                
                records_per_loc.append({'model_name': MODEL,
                               'test_set': TEST,
                                'prediction_number': i,
                                'sentence_number': all_wrong_preds[MODEL][TEST][i][0],
                                'left_context': ' '.join(tokens[:start]),
                                'right_context': ' '.join(tokens[start:]),
                                'mention_tokens': mention,
                                'predicted_labels': pred_labels,
                                'true_labels': true_labels
                               })

  0%|          | 0/39 [00:00<?, ?it/s]

KUT! 84 != 82
smth wrong, index  30
KUT
KUT! 418 != 406
smth wrong, index  49
KUT


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

KUT! 468 != 388
smth wrong, index  45
KUT
KUT! 408 != 378
smth wrong, index  66
KUT
KUT! 342 != 338
smth wrong, index  93
KUT


...

KUT! 293 != 248
smth wrong, index  10
KUT
KUT! 336 != 303
smth wrong, index  42
KUT
KUT! 468 != 318
smth wrong, index  45
KUT
KUT! 345 != 305
smth wrong, index  61
KUT
KUT! 408 != 323
smth wrong, index  66
KUT
KUT! 356 != 294
smth wrong, index  78
KUT
KUT! 326 != 289
smth wrong, index  88
KUT
KUT! 326 != 286
smth wrong, index  91
KUT
KUT! 288 != 262
smth wrong, index  92
KUT
KUT! 342 != 287
smth wrong, index  93
KUT
KUT! 350 != 288
smth wrong, index  95
KUT
KUT! 393 != 325
smth wrong, index  100
KUT


  0%|          | 0/111 [00:00<?, ?it/s]

KUT! 373 != 294
smth wrong, index  15
KUT
KUT! 229 != 228
smth wrong, index  33
KUT
KUT! 201 != 200
smth wrong, index  77
KUT
KUT! 267 != 265
smth wrong, index  91
KUT
KUT! 448 != 248
smth wrong, index  106
KUT
KUT! 284 != 262
smth wrong, index  107
KUT


  0%|          | 0/2976 [00:00<?, ?it/s]

KUT! 409 != 400
smth wrong, index  2007
KUT
KUT! 28 != 27
smth wrong, index  2434
KUT
KUT! 12 != 11
smth wrong, index  3108
KUT


In [25]:
df_per_loc = pd.DataFrame.from_records(records_per_loc)
df_per_loc.to_csv('error_analysis_df_per_loc.tsv',sep='\t')

In [26]:
df_per_loc

,model_name,test_set,prediction_number,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
0,GysBERT,test_NHA.txt,4,7,"en daar voor, de Vrouwe koperesse, by dezen te...","Zandpoort, ten dage en plaatse voor meld, in t...","[Zandpoort,]",[B-LOC],[I-PER]
1,GysBERT,test_NHA.txt,19,27,"ker, inde frankenstraat Wyk 3 Numero 782 en Ad...","A J S Muurman 38, beide te Haarlem voornoemd w...",[A],[I-LOC],[B-PER]
2,GysBERT,test_NHA.txt,23,32,enz van �1.60 38' 61} 2.21 Geregistreerd te Ha...,"Margaretha Fredrika Mab�, eenig na gelaten kin...","[Margaretha, Fredrika, Mab�,]","[B-PER, I-PER, I-PER]","[B-LOC, B-LOC, I-LOC]"
3,GysBERT,test_NHA.txt,23,32,enz van �1.60 38' 61} 2.21 Geregistreerd te Ha...,"Nicolaas Mab�, in leven No . taris, geresideer...","[Nicolaas, Mab�,]","[B-PER, I-PER]","[B-LOC, I-LOC]"
4,GysBERT,test_NHA.txt,24,34,Per Transport . . . 179�47 �ken ter somma van ...,"Pieter van Lee, in tegenwoordig heid van getui...","[Pieter, van, Lee,]","[B-PER, I-PER, I-PER]","[B-LOC, I-LOC, I-LOC]"
...,...,...,...,...,...,...,...,...,...
395,WikiNEuRal,ned.testb,2290,3891,(,FR ),[FR],[B-LOC],[B-PER]
396,WikiNEuRal,ned.testb,2537,4321,Ooit was,Compton welvarend en blank .,[Compton],[B-PER],[B-LOC]
397,WikiNEuRal,ned.testb,2542,4329,Maar dat veranderde definitief na de bloedige ...,Watts uitbraken en oversloegen op Compton .,[Watts],[B-LOC],[B-PER]
398,WikiNEuRal,ned.testb,2557,4356,""" De blanken gebruiken de latino's om",Compton terug te winnen en de zwarten eronder ...,[Compton],[B-PER],[B-LOC]


In [27]:
df_per_loc.sample(10)

,model_name,test_set,prediction_number,sentence_number,left_context,right_context,mention_tokens,predicted_labels,true_labels
40,GysBERT,test_VOC.txt,82,107,Dinsdach adij 22en Novembr Anno 1650 presentib...,"Evergem manslach te doceren, ende alsoo den se...",[Evergem],[B-LOC],[I-PER]
13,GysBERT,test_SA.txt,25,38,van de voorschreeve somma van een hon� dert da...,H: M: Hoet Emond Vandr : Voort Junior HV Heel ...,"[H:, M:, Hoet]","[B-PER, I-PER, I-PER]","[B-LOC, I-LOC, I-LOC]"
172,BERT-multi-cased,test_SA.txt,5,7,Belovende sy Compt. voor goet vast ende van wa...,"Breemen out ont. 28 Jaren, uijtgevaren geweest...",[Breemen],[I-PER],[B-LOC]
245,BERT-multi-cased,ned.testb,640,2625,,Olano ( Spa ) -- 13 .,[Olano],[B-LOC],[B-PER]
397,WikiNEuRal,ned.testb,2542,4329,Maar dat veranderde definitief na de bloedige ...,Watts uitbraken en oversloegen op Compton .,[Watts],[B-LOC],[B-PER]
279,WikiNEuRal,test_NHA.txt,27,28,"�nen deze stad, in Publieke Veiling ge, �bragt...","Nicolaas van Loon, ten Zuiden, Hendrik Lans, e...","[Nicolaas, van, Loon,]","[B-LOC, I-LOC, I-LOC]","[B-PER, I-PER, I-PER]"
209,BERT-multi-cased,ned.testb,252,1066,Egypte heeft voorgesteld dat Isra�l en de Pale...,Jeruzalem met elkaar delen zonder de eenheid v...,[Jeruzalem],[B-PER],[B-LOC]
36,GysBERT,test_VOC.txt,77,101,153 153 Van Mocha onder dato 15 Aug 1734 weder...,Turbetty deese daegen zyn niets gepasseert ont...,[Turbetty],[B-LOC],[I-PER]
278,WikiNEuRal,test_NHA.txt,25,26,Voorts gedurende den Dat voor deze Plaatsverva...,den Helder vrouw Trijntje Domper op hare quitt...,"[den, Helder]","[I-PER, I-PER]","[B-LOC, I-LOC]"
178,BERT-multi-cased,test_SA.txt,25,40,oc� tober des Jaars 1744 van Batavia aan de se...,Maccasser weduwe Carel,[Maccasser],[B-LOC],[I-PER]


In [ ]:
interesting_row_numbers = [
    106, 273,118,199,176,285,394,2,49,79,
]